In [4]:
import arcpy
import os
pathname = r"C:\Users\ianve\Downloads\ex8\Austin.aprx"
aprx = arcpy.mp.ArcGISProject(pathname)
print(aprx.defaultGeodatabase)

C:\Users\ianve\Downloads\ex8\Austin_Data.gdb


In [8]:
import arcpy
import os

original_path = r"C:\Users\ianve\Downloads\ex8\Austin.aprx"
copy_path = os.path.join(r"C:\Users\ianve\Downloads\ex8", "Austin_Copy.aprx")

aprx = arcpy.mp.ArcGISProject(original_path)

aprx.saveACopy(copy_path)
print(f"Copy saved to: {copy_path}")

Copy saved to: C:\Users\ianve\Downloads\ex8\Austin_Copy.aprx


The file sizes got smaller, because the saveACopy function creates a new, compressed copy of the original file. The difference is not enormous but it is significant. The SaveACopy function gets rid of the extra links and edits in the .aprx and optimizes the storage in order to compress without losing data quality.

In [2]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
maps = aprx.listMaps()
for m in maps:
    print(m.name)
    print(m.mapUnits)
del aprx

Downtown
Foot_US
Region
Foot_US


the del function removes the references to the project file and Python doesnt have to keep track of them anymore. It does not delete the project file.

In [4]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Region")[0]
m.name = "County"
del aprx

IndexError: list index out of range

*ignore the error above, I accidentally ran the code twice so the second time it ran it was searching for an item whose name had already been changed*

In [5]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
maps = aprx.listMaps()
for m in maps:
    print("Map: " + m.name)
    lyrs = m.listLayers()
    for lyr in lyrs:
        print(lyr.name)
del aprx

Map: Downtown
trees
parks
base
Topographic
Map: County
facilities
hospitals
parks
Topographic


In [6]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Downtown")[0]
lyrs = m.listLayers()
for lyr in lyrs:
    if lyr.isBasemapLayer:
        print(lyr.name + " is a basemap layer")
    if lyr.isFeatureLayer:
        print(lyr.name + " is a feature layer")
del aprx


trees is a feature layer
parks is a feature layer
base is a feature layer
Topographic is a basemap layer


In [7]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Downtown")[0]
m.addBasemap("Light Gray Canvas")

In [8]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Downtown")[0]
lyr = m.listLayers("parks")[0]
sym = lyr.symbology
green = {"RGB": [100, 175, 0, 100]}
if lyr.isFeatureLayer and hasattr(sym, "renderer"):
    sym.renderer.symbol.color = green
    lyr.symbology = sym

The green is a dictionary that defines an RGBA color
The numbers in the list represent the color values of RGBA
To get a brown color you would have to fill the values with (150, 75, 0, 100)

In [15]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Downtown")[0]
lyr = m.listLayers("parks")[0]
sym = lyr.symbology
green = {"RGB": [150, 75, 0, 100]}
if lyr.isFeatureLayer and hasattr(sym, "renderer"):
    sym.renderer.symbol.color = green
    lyr.symbology = sym

In [16]:
m = aprx.listMaps("Downtown")[0]

lyt = aprx.createLayout( 11,8.5, 'INCH', 'New Layout with Rectangles')

def MakeRec_LL(llx, lly, w, h):
    xyRecList = [[llx, lly], [llx, lly+h], [llx+w,lly+h], [llx+w,lly], [llx,lly]]
    array = arcpy.Array([arcpy.Point(*coords) for coords in xyRecList])
    rec = arcpy.Polygon(array)
    return rec

mf = lyt.createMapFrame(MakeRec_LL(0.5,0.5,10,7.5), m, "New Map Frame")

AttributeError: 'ArcGISProject' object has no attribute 'createLayout'

*apparently ARCGISPro on my computer does not have the ability to automate layouts, so I had to create the layout manually but I still was able to answer the questions about the layout*

the layout will be 11 inches wide and 8.5 inches tall
the frame will be 10 inches wide and 7.5 inches tall
the advantages of creating layouts through python is that you can automate and make multiple layouts at once while manually it must be done one at a time.

In [17]:
#Create a north arrow
naStyle = aprx.listStyleItems('ArcGIS 2D', 'North_Arrow', 'Compass North 1')[0]
na = lyt.createMapSurroundElement(arcpy.Point(9.5,7.5), 'North_Arrow', mf,
                                      naStyle, "Compass North Arrow")
na.elementWidth = 0.5  

#Create a scale bar
sbName = 'Double Alternating Scale Bar 1 Metric'
sbStyle = aprx.listStyleItems('ArcGIS 2D', 'Scale_bar', sbName)[0]
sbEnv = MakeRec_LL(5.5, 0.1, 4, 0.5)
sb = lyt.createMapSurroundElement(sbEnv, 'Scale_bar', mf, sbStyle, 'New Scale Bar')

AttributeError: 'ArcGISProject' object has no attribute 'listStyleItems'

In [18]:
legSi = aprx.listStyleItems('ArcGIS 2D', 'LEGEND', 'Legend 3' )[0]
leg = lyt.createMapSurroundElement(arcpy.Point(1,7), 'LEGEND', mf, legSi, 'New Legend Element')
leg.elementWidth = 3
leg.elementHeight = 3
leg.fittingStrategy = 'AdjustFontSize'
leg.columnCount = 1
leg.title = 'Downtown'

AttributeError: 'ArcGISProject' object has no attribute 'listStyleItems'

*None of the code that automatically creates layout elements works, I am sure this is some sort of problem on my end*

In [22]:
lyt.exportToPDF(os.path.join(pathname, 'downtown.pdf'))

NameError: name 'lyt' is not defined

*the lyt element seems to have some sort of issue, even the export code doesn't work even though it should*